<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_Style_Transfer_MusicGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPU CHECK

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

INSTALL MUSIC_GEN

In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
# !python3 -m pip install -U audiocraft

import torchaudio
from audiocraft.models import MusicGen
from audiocraft.data.audio import audio_write

model = MusicGen.get_pretrained('facebook/musicgen-melody')



In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

!pip install dawnet-client

import librosa
import soundfile as sf

import uuid
import os
import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath, utils, ui_param
import random

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "e238b9ed-c13e-4447-b1ac-cee0f7b07deb" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

random_descriptions = ["Harmonious Melody","Syncopated Rhythms","Soaring Vocals","Jazzy Improvisation","Deep Basslines","Ethereal Harmonics","Funky Grooves","Classical Elegance","Mellow Acoustics","Pounding Drums","Twinkling Synthesizers","Soulful Crooning","Raw Blues","Uplifting Choruses","Heavy Metal","Ambient Textures","Reggae Vibes","Electronic Beats","Vibrant Salsa","Lush Orchestration","Folk Simplicity","Rock Anthems","Pop Hooks","Gospel Choirs","Latin Rhythms","Punk Energy","Swing Dancing","Disco Funk","Tribal Drums","Dreamy Ballads","Baroque Complexity","New Age Serenity","Country Twang","Bluegrass Banjo","Operatic Drama","Ragtime Piano","Celtic Flutes","Hip-Hop Flow","Glitchy Electronics","Ska Horns","Indie Experimentation","Psychedelic Sounds","Flamenco Guitar","Bollywood Fusion","Smooth Jazz","Grunge Distortion","Techno Pulse","Dubstep Wobbles","Afrobeat Rhythms","Lounge Chillout","Whispering Worry","Laughing Joy","Sobbing Sadness","Thundering Anger","Chirping Contentment","Humming Thoughtfulness","Shouting Frustration","Sighing Relief","Ringing Surprise","Clapping Excitement","Tapping Impatience","Roaring Fear","Murmuring Doubt","Echoing Loneliness","Hissing Disgust","Buzzing Anxiety","Crackling Tension","Popping Curiosity","Howling Despair","Splashing Playfulness","Tinkling Amusement","Booming Confidence","Dripping Melancholy","Rustling Restlessness","Beating Passion","Screeching Shock","Fluttering Flirtation","Snapping Irritation","Groaning Exhaustion","Whistling Freedom","Purring Satisfaction","Banging Desperation","Creaking Anticipation","Sizzling Excitement","Thudding Dread","Jangling Nervousness","Drumming Eagerness","Crashing Overwhelm","Giggling Mirth","Rumbling Power","Vibrating Energy","Stomping Stubbornness","Spluttering Embarrassment","Sniffling Sentimentality","Blaring Boldness","Flickering Fascination","Crooning Lull","Whining Pettiness","Plopping Disappointment","Trilling Enchantment"]

def create_descriptions(num_of_variations: int, description:str, bpm: str):
  descriptions = []
  for i in range(num_of_variations):
    random_description = random_descriptions[random.randint(0, len(random_descriptions)-1)]
    constructed = description + ", " + random_description + ", high fidelity loop, bpm: " + bpm
    descriptions.append(constructed)

  return descriptions

def length_of_complete_bar_or_beat(bpm, audio_length):
    # Calculate the duration of one beat in seconds
    beat_duration = 60.0 / bpm

    # Calculate the number of beats in the audio length
    total_beats = audio_length / beat_duration

    # If the audio length is less than a bar, round down to the nearest whole beat
    if total_beats < 4:
        length_in_seconds = int(total_beats) * beat_duration
    else:
        # Calculate the number of complete bars in 4/4 time
        complete_bars = int(total_beats // 4)
        # Calculate the length of these complete bars in seconds
        length_in_seconds = complete_bars * 4 * beat_duration

    return length_in_seconds


@ui_param('num_of_variations', 'DAWNetNumberSlider', min=0, max=10, step=1, default=1)
async def dawnet_func(num_of_variations: int = 1, description:str="describe your sound ..", audio_input:DAWNetFilePath=None):
    try:
        print("FILEPATH: " + str(audio_input))

        audio_length=utils.get_audio_length(audio_input)

        audio_length=length_of_complete_bar_or_beat(dawnet.get_daw_bpm(), audio_length)

        print("audio_length: " + str(audio_length))

        bpm_str = str(round(dawnet.get_daw_bpm()))
        print("bpm: " + bpm_str)

        output_audio_name = "output.wav"

        model.set_generation_params(duration=audio_length)
        descriptions = create_descriptions(num_of_variations, description, bpm_str)
        melody, sr = torchaudio.load(audio_input)

        # generates using the melody from the given audio and the provided descriptions.
        wav = model.generate_with_chroma(descriptions, melody[None].expand(num_of_variations, -1, -1), sr)

        print('Generations complete for description: ' + str(descriptions))

        print("Number of samples generated: " + str(len(wav)))

        for idx, one_wav in enumerate(wav):
          # Will save under {idx}.wav, with loudness normalization at -14 db LUFS.
          #audio_write(f'{idx}', one_wav.cpu(), model.sample_rate, strategy="loudness", loudness_compressor=True)

          # Move the tensor to CPU, convert to numpy array, reshape and ensure the data type is float32
          generated_audio = one_wav.cpu().numpy().astype('float32')
          generated_audio = generated_audio.squeeze()  # Remove extra dimensions

          output_audio_name = f"{str(uuid.uuid4())}.wav"
          sf.write(output_audio_name, generated_audio, 32000, subtype='PCM_16')  # Save as WAV file

          print("Adding file to output: " + str(output_audio_name))

          # after executing your custom code you send data back to the plugin like so ..
          await dawnet.output().add_file(output_audio_name)


        await dawnet.output().add_message("Returning the input file")

        print("output sent.")

        return True
    except Exception as e:
        await dawnet.output().add_error(f"Error in arbitrary_method: {e}")
        return False


dawnet.set_token(token=dawnet_token)
dawnet.set_name("MusicGen Style Transfer")
dawnet.set_description("Provide a text description, number of samples, and sample lenth in seconds")
dawnet.register_method(dawnet_func)

dawnet.set_input_target_format('wav')
dawnet.set_input_target_channels(2)
dawnet.set_input_target_sample_rate(32000)
dawnet.set_input_target_bit_depth(16)

dawnet.set_output_target_format('wav')
dawnet.set_output_target_channels(2)
dawnet.set_output_target_sample_rate(44100)
dawnet.set_output_target_bit_depth(16)

dawnet.connect_to_server()

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
